# visual SDAS-Calculator

In [1]:
!pip install --force-reinstall numpy==1.26.4 matplotlib scikit-learn

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached matplotlib-3.10.1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.57.0-cp312-cp312-win_amd64.whl.metadata (104 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached pillow-11.1.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.0 requires ml_dtypes>=0.4.0, which is not installed.
jax 0.5.0 requires opt_einsum, which is not installed.
jaxlib 0.5.0 requires ml-dtypes>=0.2.0, which is not installed.
mediapipe 0.10.21 requires absl-py, which is not installed.
mediapipe 0.10.21 requires flatbuffers>=2.0, which is not installed.
mediapipe 0.10.21 requires protobuf<5,>=4.25.3, which is not installed.
mediapipe 0.10.21 requires sentencepiece, which is not installed.


In [2]:
!pip install cellpose

In [3]:
# PARAMETERS:

# Cellpose Parameters

model_type="cyto2" # can be "cyto2", "cyto3", "nuclei" or finetuned local model
diameter = 85      # is the smallest possible contour diameter

# Clustering Parameters

MIN_POINTS_PER_LINE = 5                                         # BDG-Norm: minimal number of elements in a line
MAX_POINTS_PER_LINE = 15                                        # maxmimum number of elements in a line
ANGLE_DIFF_THRESHOLD = 25                                       # maximum angle difference between current line and next line 
DISTANCE_THRESHOLD = 100                                        # maximum distance between current line and next line 
REGRESSION_DISTANCE_THRESHOLD = DISTANCE_THRESHOLD/3            # maximum distance between current regression line and next line 
MAX_ANGLE_DIFF_REG_P_THRESHOLD = 65                             # minimal angle difference between current regression line and next line 
MICROMETER_PER_PIXEL = 0.728265817023213

# Test Parameters

test_dataset_version = "bmw_data" #210 images

test_dir = f'..\data\{test_dataset_version}' 

<string>:22: SyntaxWarning: invalid escape sequence '\{'
<>:22: SyntaxWarning: invalid escape sequence '\{'
<>:22: SyntaxWarning: invalid escape sequence '\d'
<string>:22: SyntaxWarning: invalid escape sequence '\{'
<>:22: SyntaxWarning: invalid escape sequence '\{'
<>:22: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Kleinau\AppData\Local\Temp\ipykernel_8756\2188718372.py:22: SyntaxWarning: invalid escape sequence '\{'
  test_dir = f'..\data\{test_dataset_version}'
C:\Users\Kleinau\AppData\Local\Temp\ipykernel_8756\2188718372.py:22: SyntaxWarning: invalid escape sequence '\d'
  test_dir = f'..\data\{test_dataset_version}'


## Install Packages

In [4]:
from utils.my_utils import draw_contours_with_alpha, extract_line_segments, plot_line_segments_on_image, plot_line_midpoints_with_angles, group_line_segments, print_result_lines_over_img, calculate_avg_sdas, getResults, calculateMetrics 
import os
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt
from glob import glob

## Activate GPU for Cellpose (morgen erste Aufgabe)

In [ ]:
!nvcc --version
!nvidia-smi

In [ ]:
!pip uninstall torch -y

In [ ]:
!pip cache purge

In [ ]:
!pip install --force-reinstall torch==2.2.1+cu121 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import torch
print(torch.__version__)  # sollte z. B. '2.2.1+cu121' zeigen
print(torch.cuda.is_available())  # True
print(torch.cuda.get_device_name(0))  # 'NVIDIA GeForce GTX 1060 6GB'

In [ ]:
from cellpose import core, utils, io, models, metrics
use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

## Load Dendrite-Image

In [ ]:
image_name = 'T6_54_B6.jpg' 

image_path = os.path.join(test_dir, image_name)

# Load Image
img = cv2.imread(image_path)

# Show Image
plt.figure(figsize=(18,9))
plt.imshow(img)
plt.axis("off")
plt.show()

## Load Cellpose model

In [ ]:
pip install "numpy<2"

In [ ]:
import numpy

In [ ]:
print(numpy.__version__)

In [ ]:
# model_type='cyto3' oder model_type='nuclei' oder eigenes Modell

# https://cellpose.readthedocs.io/en/latest/models.html

# model = models.CellposeModel(pretrained_model='/content/drive/My Drive/SDAS_Calculator/models/CP_20250402_145747_v1')

model = models.Cellpose(gpu=True, model_type = model_type) # in jupiter Notebook geht gerade nur cyto2, scheint aber sogar besser zu sein für meinen Anwendungsfall

## Run model to get contours of dendrite structures

In [ ]:
# for cyto3/cyto2:
masks, flows, styles, imgs_dn = model.eval(img, diameter=diameter, channels= [0,0]) # Diameter ist entscheidend für sinnvolle Umrandunungen! -> kleinst möglicher Durchmesser der füllenden Linie

# for finetuned models
# masks, flows, styles = model.eval(img, diameter=diameter, channels= [0,0]) # Diameter ist entscheidend für sinnvolle Umrandunungen! -> kleinst möglicher Durchmesser der füllenden Linie

In [ ]:
img_contours = draw_contours_with_alpha(img, masks, alpha=0.75)

## Berechnung von längsten Linien durch die Konturen

In [ ]:
line_segments = extract_line_segments(masks)

In [ ]:
img_lines = plot_line_segments_on_image(img, line_segments)

## Plot Diagramm mit Mittelpunkt und Winkel

In [ ]:
plot_line_midpoints_with_angles(line_segments, img_lines)

## Clustering

In [ ]:
lines = group_line_segments(line_segments, MAX_POINTS_PER_LINE, MAX_ANGLE_DIFF_REG_P_THRESHOLD,
                         REGRESSION_DISTANCE_THRESHOLD, ANGLE_DIFF_THRESHOLD, DISTANCE_THRESHOLD,
                         MIN_POINTS_PER_LINE, MICROMETER_PER_PIXEL)

## Results

In [ ]:
print_result_lines_over_img(lines, img)

In [ ]:
print_result_lines_over_img(lines, img_lines)

In [ ]:
print_result_lines_over_img(lines, img_contours)

In [ ]:
calculate_avg_sdas(lines)

## Test on all Data and save metrics

In [ ]:
results = getResults(test_dir, model, diameter, MAX_POINTS_PER_LINE, MAX_ANGLE_DIFF_REG_P_THRESHOLD,
                                REGRESSION_DISTANCE_THRESHOLD, ANGLE_DIFF_THRESHOLD, DISTANCE_THRESHOLD,
                                MIN_POINTS_PER_LINE, MICROMETER_PER_PIXEL)

In [ ]:
SDAS_mse, SDAS_rmse, SDAS_mae, SDAS_mape, SDAS_r2 = calculateMetrics(results)

## ML-Flow 

In [ ]:
import mlflow # mlflow, version 1.23.1
import mlflow.keras
mlflow.set_experiment("SDAS_Prediciton")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

with mlflow.start_run(run_name = 'v1_HYBRID'): # change with every run 
    
    # Save hyperparameters
    
    mlflow.log_params({
        'model_type': model_type,
        'diameter': diameter,
        'MIN_POINTS_PER_LINE': MIN_POINTS_PER_LINE,
        'MAX_POINTS_PER_LINE': MAX_POINTS_PER_LINE,
        'ANGLE_DIFF_THRESHOLD': ANGLE_DIFF_THRESHOLD,
        'DISTANCE_THRESHOLD': DISTANCE_THRESHOLD,
        'REGRESSION_DISTANCE_THRESHOLD': REGRESSION_DISTANCE_THRESHOLD,
        'MAX_ANGLE_DIFF_REG_P_THRESHOLD': MAX_ANGLE_DIFF_REG_P_THRESHOLD,
        'MICROMETER_PER_PIXEL': MICROMETER_PER_PIXEL,
        'test_dataset_version': test_dataset_version 
    }) 
    
    # Save metrics
    
     mlflow.log_metrics({
        'SDAS_Mean Squared Error': SDAS_mse,
        'SDAS_Root_Mean Squared Error': SDAS_rmse,
        'SDAS_Mean Absolute Percentage Error': SDAS_mape, 
        'SDAS_Mean Absolute Error': SDAS_mae,
        'SDAS_R2-Score': SDAS_r2
    }) 
    
    # Save notebook
    
    mlflow.log_artifact("HYBRID_SDAS_Prediction_Pipeline.ipynb", artifact_path="notebooks") # Name of notebook has to be correct